In [1]:
import open3d as o3d
import numpy as np
import copy

In [2]:
target = o3d.io.read_point_cloud("./data/cloud.pcd")
source = o3d.io.read_triangle_mesh("./data/model.stl")

In [3]:
help(o3d.registration.registration_icp)

Help on built-in function registration_icp in module open3d.open3d.registration:

registration_icp(...) method of builtins.PyCapsule instance
    registration_icp(source, target, max_correspondence_distance, init=(with default value), estimation_method=registration::TransformationEstimationPointToPoint without scaling., criteria=registration::ICPConvergenceCriteria class with relative_fitness=1.000000e-06, relative_rmse=1.000000e-06, and max_iteration=30)
    
    Function for ICP registration
    
    Args:
        source (open3d.geometry.PointCloud): The source point cloud.
        target (open3d.geometry.PointCloud): The target point cloud.
        max_correspondence_distance (float): Maximum correspondence points-pair distance.
        init (numpy.ndarray[float64[4, 4]], optional): Initial transformation estimation Default value:
    
            array([[1., 0., 0., 0.],
            [0., 1., 0., 0.],
            [0., 0., 1., 0.],
            [0., 0., 0., 1.]])
        estimation_me

### mesh to cloud

In [4]:
def mesh_to_pcd(mesh, mesh_sample_number):
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_uniformly(number_of_points=mesh_sample_number)
    pcd.paint_uniform_color([1, 0.706, 0])  # yellow
    return pcd

In [6]:
source_pcd = mesh_to_pcd(source, 100000)
# o3d.visualization.draw_geometries([source])
# o3d.visualization.draw_geometries([source_pcd])

o3d.io.write_point_cloud("model.pcd", source_pcd)

True

### pick points

In [7]:
def get_picked_indices_in_cloud(cloud):
    vis = o3d.visualization.VisualizerWithEditing()

    vis.create_window()
    vis.add_geometry(cloud)
    vis.run()
    vis.destroy_window()

    return vis.get_picked_points()

In [8]:
indices_in_source = get_picked_indices_in_cloud(source_pcd)
indices_in_target = get_picked_indices_in_cloud(target)

### Convert picked indices to Open3d corresponces

In [23]:
help(o3d.utility.Vector2iVector)

Help on class Vector2iVector in module open3d.open3d.utility:

class Vector2iVector(pybind11_builtins.pybind11_object)
 |  Convert int32 numpy array of shape ``(n, 2)`` to Open3D format.
 |  
 |  Method resolution order:
 |      Vector2iVector
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bool__(...)
 |      __bool__(self: open3d.open3d.utility.Vector2iVector) -> bool
 |      
 |      Check whether the list is nonempty
 |  
 |  __contains__(...)
 |      __contains__(self: open3d.open3d.utility.Vector2iVector, x: numpy.ndarray[int32[2, 1]]) -> bool
 |      
 |      Return true the container contains ``x``
 |  
 |  __copy__(...)
 |      __copy__(self: open3d.open3d.utility.Vector2iVector) -> open3d.open3d.utility.Vector2iVector
 |  
 |  __deepcopy__(...)
 |      __deepcopy__(self: open3d.open3d.utility.Vector2iVector, arg0: dict) -> open3d.open3d.utility.Vector2iVector
 |  
 |  __delitem__(...)
 |      __delitem__(*args, **kwa

In [9]:
corres_indices = np.zeros((len(indices_in_source), 2))
corres_indices[:, 0] = indices_in_source
corres_indices[:, 1] = indices_in_target
correspondences = o3d.utility.Vector2iVector(corres_indices)

print(indices_in_source)
print(indices_in_target)
print("")
print(corres_indices)
print("")
print(correspondences)


[97146, 71348, 89565, 92943, 97859, 70429, 88722, 92378]
[155580, 120005, 99324, 77720, 155322, 123175, 101721, 80592]

[[ 97146. 155580.]
 [ 71348. 120005.]
 [ 89565.  99324.]
 [ 92943.  77720.]
 [ 97859. 155322.]
 [ 70429. 123175.]
 [ 88722. 101721.]
 [ 92378.  80592.]]

std::vector<Eigen::Vector2i> with 8 elements.
Use numpy.asarray() to access data.


### point to point transform

In [16]:
help(o3d.registration.TransformationEstimationPointToPoint)

Help on class TransformationEstimationPointToPoint in module open3d.open3d.registration:

class TransformationEstimationPointToPoint(TransformationEstimation)
 |  Class to estimate a transformation for point to point distance.
 |  
 |  Method resolution order:
 |      TransformationEstimationPointToPoint
 |      TransformationEstimation
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __copy__(...)
 |      __copy__(self: open3d.open3d.registration.TransformationEstimationPointToPoint) -> open3d.open3d.registration.TransformationEstimationPointToPoint
 |  
 |  __deepcopy__(...)
 |      __deepcopy__(self: open3d.open3d.registration.TransformationEstimationPointToPoint, arg0: dict) -> open3d.open3d.registration.TransformationEstimationPointToPoint
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: open3d.open3d.registration.TransformationEstimationPointToPoint, ar

In [10]:
p2p = o3d.registration.TransformationEstimationPointToPoint()
init_transform = p2p.compute_transformation(source_pcd, target, correspondences)
print(init_transform)

[[-9.99250894e-01  3.86682045e-02  1.55618053e-03  1.81261269e+00]
 [ 2.17521618e-02  5.27944595e-01  8.49000205e-01 -1.11115109e+00]
 [ 3.20077364e-02  8.48398064e-01 -5.28390225e-01  4.49769626e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target])

In [12]:
draw_registration_result(source_pcd, target, init_transform)

In [224]:
threshold = 0.02
reg_p2p = o3d.registration.registration_icp(
        source_pcd, target, threshold, init_transform,
        o3d.registration.TransformationEstimationPointToPoint())
print(reg_p2p.transformation)

draw_registration_result(source_pcd, target, reg_p2p.transformation)

[[-0.99879802  0.04861176  0.00627801  1.80355423]
 [ 0.03132713  0.53459318  0.84452871 -1.09027537]
 [ 0.03769785  0.84371028 -0.53547347  4.49124158]
 [ 0.          0.          0.          1.        ]]


In [225]:
source_temp = copy.deepcopy(source_pcd)
source_temp.transform(reg_p2p.transformation)
o3d.io.write_point_cloud("tfed_model.pcd", source_temp)

True

In [226]:
from pyquaternion import Quaternion

def matrix2qua(matrix):
    qua = Quaternion(matrix=matrix[0:3, 0:3])
    return qua.elements


def matrix2trans(matrix):
    return np.array(matrix[:3, 3:].transpose())[0]


def matrix2pose(matrix):
    qua = matrix2qua(matrix)
    translation = matrix2trans(matrix)
    return np.hstack((translation, qua))

In [338]:
print(reg_p2p.transformation)
print("")
print(matrix2pose(reg_p2p.transformation))
print("")

rotation = reg_p2p.transformation[:3, :3]
translation = reg_p2p.transformation[:3, 3:].transpose()[0]
print(rotation)
print("")
print(translation)

[[-0.99879802  0.04861176  0.00627801  1.80355423]
 [ 0.03132713  0.53459318  0.84452871 -1.09027537]
 [ 0.03769785  0.84371028 -0.53547347  4.49124158]
 [ 0.          0.          0.          1.        ]]

[ 1.80355423 -1.09027537  4.49124158 -0.00896778  0.02281599  0.87590877
  0.48185355]

[[-0.99879802  0.04861176  0.00627801]
 [ 0.03132713  0.53459318  0.84452871]
 [ 0.03769785  0.84371028 -0.53547347]]

[ 1.80355423 -1.09027537  4.49124158]


In [272]:
l,w,h = [0.6, 0.4, 0.2]

front_bottom = np.array([0.275, 0.021, 0.44])
back_bottom = np.array([0.275, 0.621, 0.44])

center_in_model = (front_in_model + back_in_model) / 2 + np.array([0,0,h/2])

points_in_model = [
    front_bottom,                           # ref pt
    back_bottom,                            # pt to x
    front_bottom + np.array([-w/2, 0, 0]),  # pt to y
    front_bottom + np.array([0, 0, h])      # pt to z
]

[0.275 0.021 0.44 ]
<class 'numpy.ndarray'>
[0.275 0.321 0.54 ]
<class 'numpy.ndarray'>


In [339]:
def calculate_rotation_matrix(points):
    x_axis = points[1] - points[0]
    x_axis = x_axis / np.linalg.norm(x_axis)

    y_axis = points[2] - points[0]
    y_axis = y_axis / np.linalg.norm(y_axis)

    z_axis = points[3] - points[0]
    z_axis = z_axis / np.linalg.norm(z_axis)
    
    return np.vstack((x_axis, y_axis, z_axis)).transpose()


def calculate_transform_matrix(center, points):
    rotation_matrix = calculate_rotation_matrix(points)
    transform_matrix = np.hstack((rotation_matrix, np.array([center]).transpose()))
    transform_matrix = np.vstack((transform_matrix, np.mat([[0, 0, 0, 1]])))
    
    return transform_matrix

In [316]:
compartment_matrix_in_model = calculate_matrix(center_in_model, points_in_model)
compartment_pose_in_model = matrix2pose(compartment_matrix_in_model)
print(compartment_matrix_in_model)
print(compartment_pose_in_model)
print("")

compartment_matrix_in_cloud = np.dot(compartment_matrix_in_model, reg_p2p.transformation)
compartment_pose_in_cloud = matrix2pose(compartment_matrix_in_cloud)
print(compartment_matrix_in_cloud)
print(compartment_pose_in_cloud)
print("")

[[ 0.    -1.     0.     0.275]
 [ 1.     0.     0.     0.321]
 [ 0.     0.     1.     0.54 ]
 [ 0.     0.     0.     1.   ]]
[0.275      0.321      0.54       0.70710678 0.         0.
 0.70710678]

[[-0.03132713 -0.53459318 -0.84452871  1.36527537]
 [-0.99879802  0.04861176  0.00627801  2.12455423]
 [ 0.03769785  0.84371028 -0.53547347  5.03124158]
 [ 0.          0.          0.          1.        ]]
[ 1.36527537  2.12455423  5.03124158 -0.34706309 -0.60322769  0.63549437
  0.33438073]



In [321]:
shlef_in_model = [
    np.array([0,0,0]),
    np.array([0,1,0]),
    np.array([-1,0,0]),
    np.array([0,0,1])
]

shlef_in_cloud = []
for point in shlef_in_model:
    shlef_in_cloud.append(np.dot(point, rotation) + translation)

shelf_matrix_in_cloud = calculate_matrix(shlef_in_cloud[0], shlef_in_cloud)
shelf_pose_in_cloud = matrix2pose(shelf_matrix_in_cloud)

temp_compartment_matrix = np.array(
[[1,  0,   0.,  0.321],
 [ 0, 1,   0. ,    -0.275],
 [ 0.  ,   0. ,    1.,     0.54 ],
 [ 0. ,    0. ,    0.,     1.   ]])

temp = np.dot(shelf_matrix_in_cloud, temp_compartment_matrix)
temp_pose = matrix2pose(temp)
print(temp)
print(temp_pose)

print(type(shlef_in_model))

[[ 0.03132713  0.99879802  0.03769785  1.55929762]
 [ 0.53459318 -0.04861176  0.84371028 -0.44969918]
 [ 0.84452871 -0.00627801 -0.53547347  4.47490608]
 [ 0.          0.          0.          1.        ]]
[ 1.55929762 -0.44969918  4.47490608 -0.33438073  0.63549437  0.60322769
  0.34706309]
<class 'list'>


In [340]:
shlef_in_model = [
    np.array([0,0,0]),
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

shlef_in_cloud = []
for point in shlef_in_model:
    shlef_in_cloud.append(np.dot(point, rotation) + translation)

shelf_matrix_in_cloud = calculate_transform_matrix(shlef_in_cloud[0], shlef_in_cloud)
shelf_pose_in_cloud = matrix2pose(shelf_matrix_in_cloud)

temp = np.dot(shelf_matrix_in_cloud, compartment_matrix_in_model)
temp_pose = matrix2pose(temp)
print(temp)
print(temp_pose)

[[ 0.03132713  0.99879802  0.03769785  1.55929762]
 [ 0.53459318 -0.04861176  0.84371028 -0.44969918]
 [ 0.84452871 -0.00627801 -0.53547347  4.47490608]
 [ 0.          0.          0.          1.        ]]
[ 1.55929762 -0.44969918  4.47490608 -0.33438073  0.63549437  0.60322769
  0.34706309]


In [275]:
center_in_cloud = np.dot(center_in_model, rotation) + translation
points_in_cloud = []
for point in points_in_model:
    points_in_cloud.append(np.dot(point, rotation) + translation)

compartment_matrix_in_cloud_directly = calculate_matrix(center_in_cloud, points_in_cloud)
compartment_pose_in_cloud_directly = matrix2pose(compartment_matrix_in_cloud_directly)
print(compartment_matrix_in_cloud_directly)
print(compartment_pose_in_cloud_directly)

[[ 0.03132713  0.99879802  0.03769785  1.55929762]
 [ 0.53459318 -0.04861176  0.84371028 -0.44969918]
 [ 0.84452871 -0.00627801 -0.53547347  4.47490608]
 [ 0.          0.          0.          1.        ]]
[ 1.55929762 -0.44969918  4.47490608 -0.33438073  0.63549437  0.60322769
  0.34706309]
